In [1]:
import pandas as pd
from exhibit import exhibit as xbt
from exhibit.core.spec import Spec, UUIDColumn, CategoricalColumn, NumericalColumn, DateColumn

from exhibit.db import db_util

### Using external tables to generate conditional categorical and date values
You can write SQL statements in the `anonymising_set` field to define the set of values that will be used for this column. SQL has to start with `SELECT` and must include fully qualified join keys and the target column aliased to the column name. That way, Exhibit will know how to join back the values from the Exhibit database. You can mix external columns with already generated columns to have complete flexibility in determining inter-column relationships. Please note that the DB bundled with Exhibit is using the SQLite dialect so adjust your SQL if using a different dialect.

#### Conditional categorical columns

In [2]:
# Here, we define a list of conditions that are gender-specific.
# If you don't include the probabilities for a condition in the specification, the probabilities will be taken from the table.
linked_data = pd.DataFrame(data={
    "gender"   : ["M", "M", "M", "F", "F", "F"],
    "condition": ["A", "B", "C", "C", "D", "E"]
})

# We'll insert this conditonal table into the Exhibit DB, but you can use any of the existing tables if using your own DB
db_util.insert_table(linked_data, "temp_linked")

Successfully inserted a new table temp_linked


In [3]:
spec = Spec()
spec_dict = spec.generate()

spec_dict["metadata"]["number_of_rows"] = 100
spec_dict["metadata"]["categorical_columns"] = ["gender", "condition"]
spec_dict["metadata"]["numerical_columns"] = ["count"]
spec_dict["metadata"]["id"] = "main"

# You can specify custom probabilities and weights for numerical columns,
# just like you would for a standard categorical column
condition_data = pd.DataFrame(data={
    "condition"          : ["A", "B", "C", "D", "E", "Missing Data"],
    "probability_vector" : [0.1, 0.1, 0.5, 0.1, 0.2, 0.0],
    "count"              : [0.1, 0.1, 0.1, 0.1, 0.6, 0.0],
})

# Note that in order to reference the gender column that is added to the spec before condition, we 
# use temp_main table name, which references the id from the metadata section of the spec.
# All non-aliased columns will be used as join keys.
condition_sql = """
SELECT temp_main.gender, temp_linked.condition as condition
FROM temp_main JOIN temp_linked ON temp_main.gender = temp_linked.gender
"""

gender_data = pd.DataFrame(data={
    "gender" : ["M", "F", "Missing Data"],
    "probability_vector" : [0.5, 0.5, 0],
})


spec_dict["columns"]["gender"]    = CategoricalColumn("gender", uniques=2, original_values=gender_data, anon_set="random")
spec_dict["columns"]["condition"] = CategoricalColumn("condition", uniques=5, original_values=condition_data, anon_set=condition_sql)
spec_dict["columns"]["count"]     = NumericalColumn(distribution="weighted_uniform", distribution_parameters={"target_sum": 50_000, "dispersion":0.2})

exhibit_data = xbt.Exhibit(command="fromspec", source=spec_dict, output="dataframe")
anon_df = exhibit_data.generate()

In [4]:
anon_df.head()

,gender,condition,count
0,F,C,265
1,M,C,250
2,M,A,305
3,F,C,250
4,F,E,1761


In [5]:
# clean up the temp_tables
db_util.purge_temp_tables()

Successfully deleted 2 tables


#### Conditional date columns
Exhibit DB includes a dates column that has a full range of dates at a day level between `01/01/1900` and `01/01/2100`. These can be used to generate conditional dates, such as dates of death for each patient given the randomly generated date of birth.

In [6]:
spec = Spec()
spec_dict = spec.generate()

spec_dict["metadata"]["number_of_rows"] = 100
spec_dict["metadata"]["uuid_columns"] = ["patient"]
spec_dict["metadata"]["date_columns"] = ["date_of_birth", "date_of_death"]
spec_dict["metadata"]["id"] = "main"

# Note that because for each patient the range of possible dates to pick is quite large, the generation will take longer than
# if you were using a basic constraint to ensure one date (or numerical value) is always greater than another. The SQL approach,
# however, is much more flexible and results in a more realistic-looking dataset.
date_sql = """
SELECT temp_main.patient, dates.date as date_of_death
FROM temp_main JOIN dates
WHERE dates.date >= temp_main.date_of_birth AND dates.date < '2023-01-01'
"""

# Note that we're using pseudo_chi here as identifier - the numbers are randomised and the month is set to 13 to avoid accidental collisions
spec_dict["columns"]["patient"]       = UUIDColumn(uuid_seed=0, anon_set="pseudo_chi")
spec_dict["columns"]["date_of_birth"] = DateColumn("date_of_birth", uniques=80, from_date="1980-01-01", to_date="2023-01-01", cross_join=False)
spec_dict["columns"]["date_of_death"] = DateColumn("date_of_death", uniques=80, anonymising_set=date_sql, cross_join=False, miss_proba=0.8)

exhibit_data = xbt.Exhibit(command="fromspec", source=spec_dict, output="dataframe")
anon_df = exhibit_data.generate()

In [7]:
anon_df.head(10)

,patient,date_of_birth,date_of_death
0,1013281814,2007-12-08,NaT
1,2613691125,2020-01-22,2021-06-06
2,0013543945,2002-07-22,NaT
3,1713966817,2020-10-20,2021-12-04
4,2213686955,2000-08-19,NaT
5,2913847760,1980-05-09,NaT
6,0513956112,2018-03-29,2021-03-27
7,2213457598,1987-07-13,NaT
8,1713855054,1983-10-30,NaT
9,3013491978,2017-08-06,NaT


In [8]:
# clean up the temp_tables
db_util.purge_temp_tables()

Successfully deleted 1 tables
